In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
import os
from tqdm import tqdm
from dotenv import load_dotenv
from roboflow import Roboflow

In [ ]:
load_dotenv()

In [3]:
transform = ToTensor()

In [ ]:
rf = Roboflow(api_key=os.getenv("API_KEY"))
project = rf.workspace(os.getenv("WORKSPACE")).project(os.getenv("PROJECT"))
version = project.version(os.getenv("VERSION"))
dataset = version.download(os.getenv("DATASET_FORMAT"))

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Tempat-Sampah-Pintar-1 in coco:: 100%|██████████| 6028/6028 [00:01<00:00, 4227.73it/s]


In [5]:
class CocoDetectionCustom(CocoDetection):
    def __getitem__(self, idx):
        img, target = super(CocoDetectionCustom, self).__getitem__(idx)

        boxes = []
        labels = []

        for obj in target:
            xmin = obj['bbox'][0]
            ymin = obj['bbox'][1]
            xmax = xmin + obj['bbox'][2]
            ymax = ymin + obj['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(obj['category_id'])

        if len(boxes) == 0:
            boxes = torch.empty((0, 4), dtype=torch.float32)
            labels = torch.empty((0,), dtype=torch.int64)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}

        return img, target

In [6]:
dataset_dir = "/content/Tempat-Sampah-Pintar-1"

train_dir = os.path.join(dataset_dir, "train")
val_dir = os.path.join(dataset_dir, "valid")

In [7]:
train_dataset = CocoDetectionCustom(root=train_dir, annFile=os.path.join(train_dir, "_annotations.coco.json"), transform=transform)
val_dataset = CocoDetectionCustom(root=val_dir, annFile=os.path.join(val_dir, "_annotations.coco.json"), transform=transform)


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [8]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))


In [9]:
num_classes = len(train_dataset.coco.cats) + 1
model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 193MB/s]


In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
num_epochs = 5

In [12]:

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    epoch_iterator = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]", unit="batch")

    for images, targets in epoch_iterator:
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        train_loss += losses.item()

        epoch_iterator.set_postfix(loss=(train_loss / (epoch_iterator.n + 1)))

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss/len(train_loader)}")


torch.save(model.state_dict(), "faster_rcnn_coco.pth")
print("Training complete, model saved as 'faster_rcnn_coco.pth'")


Epoch [1/5]: 100%|██████████| 1094/1094 [02:42<00:00,  6.75batch/s, loss=0.271]


Epoch [1/5], Loss: 0.27056227759262325


Epoch [2/5]: 100%|██████████| 1094/1094 [02:39<00:00,  6.88batch/s, loss=0.194]


Epoch [2/5], Loss: 0.1944549884715235


Epoch [3/5]: 100%|██████████| 1094/1094 [02:39<00:00,  6.87batch/s, loss=0.166]


Epoch [3/5], Loss: 0.16581779385029835


Epoch [4/5]: 100%|██████████| 1094/1094 [02:39<00:00,  6.87batch/s, loss=0.147]


Epoch [4/5], Loss: 0.1471415073415043


Epoch [5/5]: 100%|██████████| 1094/1094 [02:39<00:00,  6.86batch/s, loss=0.133]


Epoch [5/5], Loss: 0.1334333497131928
Training complete, model saved as 'faster_rcnn_coco.pth'
